## Import Packages

In [11]:
import cv2
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
import time
import cv2
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import normalize
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import RidgeClassifierCV

## Load training/testing Data

In [12]:
# Load images first
numImages = len(glob.glob('./images/*jpg'))
images = [None for i in xrange(numImages)]
for fileName in glob.glob('./images/*jpg'):
    fileNum = int(fileName[9:][:-4])
    images[fileNum-1] = np.array(cv2.imread(fileName, 0))
images = np.array(images)

# Load csv data next
train_data = pd.read_csv('data/train.csv').drop(['species'], axis=1).values
train_labels = pd.read_csv('data/train.csv')['species'].values
train_images = [images[int(data[0]-1)] for data in train_data]
train_ids = [data[0] for data in train_data]
train_data = np.delete(train_data, 0, 1)


test_data = pd.read_csv('data/test.csv').values
test_images = [images[int(data[0]-1)] for data in test_data]
test_ids = [data[0] for data in test_data]
test_data = np.delete(test_data, 0, 1)


del images

## Separate the feature-vectors

In [13]:

#separate the 3 histograms
# train_margin_data=((pd.read_csv('data/train.csv').drop(['species'], axis=1)).loc[:,'margin1':'margin64']).values
# train_shape_data=((pd.read_csv('data/train.csv').drop(['species'], axis=1)).loc[:,'shape1':'shape64']).values
# train_texture_data=((pd.read_csv('data/train.csv').drop(['species'], axis=1)).loc[:,'texture1':'texture64']).values

# test_margin_data=((pd.read_csv('data/test.csv')).loc[:,'margin1':'margin64']).values
# test_shape_data=((pd.read_csv('data/test.csv')).loc[:,'shape1':'shape64']).values
# test_texture_data=((pd.read_csv('data/test.csv')).loc[:,'texture1':'texture64']).values


train_data = pd.read_csv('data/train.csv').drop(['species'], axis=1).values
print train_data.shape
test_data = pd.read_csv('data/test.csv').values
print test_data.shape
test_ids = [data[0] for data in test_data]
print len(test_ids)
train_labels = pd.read_csv('data/train.csv')['species'].values
print len(train_labels)
train_ids = [data[0] for data in train_data]

le= preprocessing.LabelEncoder()
#encode train labels
le.fit(train_labels)
train_labels_encoded=le.transform(train_labels)


(990, 193)
(594, 193)
594
990


## Build, Train, and Test the Ridge Classifier Model with Cross Validation

In [20]:
classifier = RidgeClassifierCV()

In [21]:
classifier.fit(train_data,train_labels_encoded)

RidgeClassifierCV(alphas=(0.1, 1.0, 10.0), class_weight=None, cv=None,
         fit_intercept=True, normalize=False, scoring=None)

In [24]:
final_pred = classifier.predict(test_data)
final_confidence =  np.abs(classifier.decision_function(test_data))
final_confidence = np.append(np.array(test_ids).reshape(-1,1),final_confidence,axis=1)

In [25]:

headerRow=np.array(['id'] + le.inverse_transform(range(99)).tolist())
df = pd.DataFrame(data=final_confidence, columns = headerRow)
df['id'] = df['id'].astype(np.int)
df=df.set_index('id')
df.to_csv('output/18_12_18_01.csv')